# Задание 1

Выполнено на мощностях [Google Colab](https://colab.research.google.com/drive/1JDr378iGS0og3KZs1-tPaS2QlWmi45Ue)

## Подготовить мини-корпус (4-5 текстов или до 10 тысяч токенов) с разметкой ключевых слов. Желательно указать источник корпуса и описать, в каком виде там были представлены ключевые слова.

Возьмём стандарт индустрии для задачи определения ключевых слов -- корпус **Hulth (2003)** на английском языке. В удобном предобработанном виде его можно взять [отсюда](https://github.com/boudinfl/hulth-2003-pre).

In [1]:
!git clone https://github.com/boudinfl/hulth-2003-pre

fatal: destination path 'hulth-2003-pre' already exists and is not an empty directory.


Вот как выглядит пример документа в корпусе:

In [2]:
!cat hulth-2003-pre/train/100.xml

<?xml version="1.0" encoding="UTF-8"?>
<?xml-stylesheet href="CoreNLP-to-HTML.xsl" type="text/xsl"?>
<root>
  <document>
    <sentences>
      <sentence id="1">
        <tokens>
          <token id="1">
            <word>Separate</word>
            <lemma>separate</lemma>
            <CharacterOffsetBegin>0</CharacterOffsetBegin>
            <CharacterOffsetEnd>8</CharacterOffsetEnd>
            <POS>JJ</POS>
          </token>
          <token id="2">
            <word>accounts</word>
            <lemma>account</lemma>
            <CharacterOffsetBegin>9</CharacterOffsetBegin>
            <CharacterOffsetEnd>17</CharacterOffsetEnd>
            <POS>NNS</POS>
          </token>
          <token id="3">
            <word>go</word>
            <lemma>go</lemma>
            <CharacterOffsetBegin>18</CharacterOffsetBegin>
            <CharacterOffsetEnd>20</CharacterOffsetEnd>
            <POS>VBP</POS>
          </token>
          <token id="4">
            <word>mainstream</word>
       

Напишем функцию для приведения такого документа к удобному нам формату:

In [0]:
from bs4 import BeautifulSoup

In [0]:
def getHulthDoc(filepath):
  with open(filepath, "r", encoding="utf-8") as inh:
    xmlstr = inh.read()
  soup = BeautifulSoup(xmlstr)
  doc = [[str(l.text).lower() for l in sentence.findAll("lemma")]
         for sentence in soup.root.document.sentences.findAll("sentence")]
  return doc

In [5]:
print(getHulthDoc("hulth-2003-pre/train/100.xml"))

[['separate', 'account', 'go', 'mainstream', '-lsb-', 'investment', '-rsb-'], ['new', 'entrant', 'be', 'shake', 'up', 'the', 'separate-account', 'industry', 'by', 'supply', 'web-based', 'platform', 'that', 'give', 'adviser', 'the', 'tool', 'to', 'pick', 'independent', 'money', 'manager']]


## Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.

Выберем подкорпус так, чтобы в нём было чуть менее 10000 токенов

In [0]:
import os
import random
from itertools import chain

random.seed(42)

In [0]:
def getCorpusLen(corpus):
  return len(tuple(chain.from_iterable(chain.from_iterable(corpus))))

In [8]:
source_files = sorted(list(os.listdir("hulth-2003-pre/train")))
random.shuffle(source_files)
selected = []
Corpus = []
max_len = 10000
i = 0

while getCorpusLen(Corpus) < max_len:
  fname = source_files[i]
  if fname in selected:
    continue
  Corpus.append(getHulthDoc("hulth-2003-pre/train/"+fname))
  selected.append(fname)
  print(str(i+1)+".\t"+fname)
  i += 1

Corpus = Corpus[:-1]
selected = selected[:-1]

1.	797.xml
2.	544.xml
3.	903.xml
4.	928.xml
5.	1031.xml
6.	1445.xml
7.	108.xml
8.	771.xml
9.	1323.xml
10.	559.xml
11.	1169.xml
12.	826.xml
13.	1382.xml
14.	641.xml
15.	709.xml
16.	539.xml
17.	780.xml
18.	1047.xml
19.	755.xml
20.	1305.xml
21.	1415.xml
22.	809.xml
23.	1318.xml
24.	1430.xml
25.	648.xml
26.	1439.xml
27.	640.xml
28.	587.xml
29.	1334.xml
30.	791.xml
31.	655.xml
32.	944.xml
33.	815.xml
34.	1458.xml
35.	93.xml
36.	1338.xml
37.	624.xml
38.	744.xml
39.	1050.xml
40.	1391.xml
41.	1053.xml
42.	1240.xml
43.	793.xml
44.	117.xml
45.	143.xml
46.	808.xml
47.	116.xml
48.	1138.xml
49.	763.xml
50.	1104.xml
51.	1393.xml
52.	968.xml
53.	1017.xml
54.	1173.xml
55.	1379.xml
56.	618.xml
57.	695.xml
58.	783.xml
59.	1201.xml
60.	615.xml
61.	610.xml
62.	1272.xml
63.	663.xml
64.	1353.xml
65.	1333.xml
66.	89.xml
67.	1303.xml
68.	1015.xml
69.	1185.xml
70.	1000.xml


Ого, здесь около 70 файлов. Многовато! Поэтому доверимся корпусной разметке и пожертвуем уж двумя баллами за этот пункт. Самое время загрузить golden dataset:

In [0]:
import json

In [0]:
with open("hulth-2003-pre/references/train.uncontr.json", "r") as injson:
  Data = json.load(injson)

GoldenDataset = [list(chain.from_iterable([token.split(" ") for token in list(chain.from_iterable(Data[fname[:-4]]))])) for fname in selected]

Всё же немного изменим образец разметки: подавляющее количесво ключевых слов в нём -- n-граммы. Наши же методы не имеют такой большой склонности к ним, а tf-idf вообще умеет выделять только единичные токены, так что разделим токены дополнительно по пробелу. Так как мы учитываем повторение слов при нашем подсчёте, то потеряем точность мы только на вхождениях с неверным порядком слов (что редко) и вхождениях из одного слова n-граммы (и не факт, что это хуже).

In [11]:
GoldenDataset[0]

['adaptive',
 'wavelet',
 'methods',
 'elliptic',
 'case',
 'operator',
 'equations',
 'least',
 'squares',
 'formulation',
 'euclidean',
 'metric',
 'asymptotically',
 'optimal',
 'complexity',
 'n-term',
 'approximation']

## Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25).

Поставим таггеры в равные условия, предоставив им одинаковый набор стоп-слов.

In [0]:
import nltk

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
stop = stopwords.words('english')

И посчитаем всё!

### RAKE

In [15]:
!pip install python-rake

In [0]:
import RAKE

In [0]:
rake = RAKE.Rake(stop)

In [0]:
def processRake(doc):
  res = rake.run(" ".join([" ".join(sent) for sent in doc]), maxWords=3, minFrequency=2)
  return [pair[0] for pair in res]

In [0]:
RAKE_kw = [processRake(doc) for doc in Corpus]

Так как RAKE умеет выдавать токены из нескольких слов, а мы решили работать с однословными, такие вхождения придётся разделить:

In [0]:
RAKE_kw = [list(chain.from_iterable([el.split(" ") for el in token])) for token in RAKE_kw]

### TextRank

In [21]:
!pip install summa

In [0]:
from summa import keywords

In [0]:
def processTextRank(doc):
  res = keywords.keywords(" ".join([" ".join(sent) for sent in doc]), additional_stopwords=stop, scores=True)
  return [pair[0] for pair in res]

In [0]:
TextRank_kw = [processTextRank(doc) for doc in Corpus]

TextRank тоже умеет в многословные токены - исправляем:

In [0]:
TextRank_kw = [list(chain.from_iterable([el.split(" ") for el in token])) for token in TextRank_kw]

### tf-idf

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
vectorizer = TfidfVectorizer(stop_words=stop)
transformed = vectorizer.fit_transform([" ".join([" ".join(sent) for sent in doc]) for doc in Corpus])

In [0]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [0]:
topn = 5

feature_names = vectorizer.get_feature_names()
tfidf_kw = []

for tf_idf_vector in transformed:
  sorted_items = sort_coo(tf_idf_vector.tocoo())
  keywords = extract_topn_from_vector(feature_names,sorted_items,topn)
  tfidf_kw.append(list(keywords.keys()))

## Оценить точность, полноту, F-меру выбранных методов относительно имеющейся разметки.

In [0]:
import numpy as np
import pandas as pd

from copy import deepcopy
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

Сделаем предобработку для списков, чтобы можно было использовать функции для подсчёта метрик качества из библиотеки sklearn:

In [0]:
def preprocess(true, pred):
  true = deepcopy(true)
  pred = deepcopy(pred)
  total = true + pred
  y_true = [0] * len(total)
  y_pred = [0] * len(total)
  for i, el in enumerate(total):
    for j, tel in enumerate(true):
      if tel == el:
        true[j] = None
        y_true[i] = 1
        break
    for j, pel in enumerate(pred):
      if pel == el:
        pred[j] = None
        y_pred[i] = 1
        break
  del_idx = []
  for i, _ in enumerate(total):
    if not y_true[i] and not y_pred[i]:
      del_idx.append(i)
  for i in reversed(sorted(del_idx)):
    del y_true[i]
    del y_pred[i]
  return y_true, y_pred

In [32]:
preprocess(["a", "b", "d", "a"], ["a", "c", "e", "d", "a"])

([1, 1, 1, 1, 0, 0], [1, 0, 1, 1, 1, 1])

Сделаем красиво: посчитаем средние precision, recall и F1 по корпусу для всех трёх методов выделения ключевых слов и представим их в процентном виде (умножим на 100 и округлим до двух знаков после запятой)

In [0]:
def precision_single(true, pred):
  y_true, y_pred = preprocess(true, pred)
  return precision_score(y_true, y_pred)

def recall_single(true, pred):
  y_true, y_pred = preprocess(true, pred)
  return recall_score(y_true, y_pred)

def f1_single(true, pred):
  y_true, y_pred = preprocess(true, pred)
  return f1_score(y_true, y_pred)

In [0]:
def precision(list_of_trues, list_of_preds):
  tr = list_of_trues
  pr = list_of_preds
  score = np.mean([precision_single(tr[i], pr[i]) for i in range(len(tr))])
  score = round(score * 100, 2)
  return score

def recall(list_of_trues, list_of_preds):
  tr = list_of_trues
  pr = list_of_preds
  score = np.mean([recall_single(tr[i], pr[i]) for i in range(len(tr))])
  score = round(score * 100, 2)
  return score

def f1(list_of_trues, list_of_preds):
  tr = list_of_trues
  pr = list_of_preds
  score = np.mean([f1_single(tr[i], pr[i]) for i in range(len(tr))])
  score = round(score * 100, 2)
  return score

Наконец, подсчитаем всё и сведём в табличку:

In [35]:
Result = [
          ["RAKE", precision(GoldenDataset, RAKE_kw), recall(GoldenDataset, RAKE_kw), f1(GoldenDataset, RAKE_kw)],
          ["TextRank", precision(GoldenDataset, TextRank_kw), recall(GoldenDataset, TextRank_kw), f1(GoldenDataset, TextRank_kw)],
          ["tf-idf", precision(GoldenDataset, tfidf_kw), recall(GoldenDataset, tfidf_kw), f1(GoldenDataset, tfidf_kw)]
]

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [36]:
df = pd.DataFrame.from_records(Result)
df.columns = ["Method", "Precision", "Recall", "F1 score"]

df

,Method,Precision,Recall,F1 score
0,RAKE,33.37,7.70,11.35
1,TextRank,48.23,17.33,24.43
2,tf-idf,60.00,16.35,23.85


## Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.